<a href="https://colab.research.google.com/github/aivan2798/01/blob/main/smartbokx_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch

In [6]:
notes_data = pd.read_csv('/content/drive/MyDrive/notes/memory_1722173127335.csv')
notes_data.tail()

,note_id,note_index,note_title,note_link,note_description
138,5,0,https://vm.tiktok.com/ZMMXJxj8q/,https://vm.tiktok.com/ZMMXJxj8q/,Jesus as a black African
139,4,0,https://vm.tiktok.com/ZMMCoEnkN/,https://vm.tiktok.com/ZMMCoEnkN/,"No time to waste, any money you see, you thief..."
140,3,0,https://www.researchgate.net/publication/33315...,https://www.researchgate.net/publication/33315...,Research gate notes on input layers and CNN mo...
141,2,0,https://vm.tiktok.com/ZMMCrb6xv/,https://vm.tiktok.com/ZMMCrb6xv/,Sadnguru talking about heaven and society
142,1,0,https://t.me/friendsforever265,https://t.me/friendsforever265,Link to the friends forever telegram group wit...


In [9]:
note_titles = notes_data['note_title']
note_links = notes_data['note_link']
note_descriptions = notes_data['note_description']
note_descriptions.head()

0                               Mars rover deployment 
1                        White horse in paris olympics
2    Why everone thought that 2012 was the end of t...
3                      Your cells are fighting for you
4                      Bird helping rat cross the road
Name: note_description, dtype: object

In [16]:
def genBatchTokens(text):
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  tokens = tokenizer.batch_encode_plus(text, padding=True,add_special_tokens=True,truncation=True,return_tensors = "pt")
  return tokens

def genToken(text):
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  tokens = tokenizer.encode_plus(text, padding=True,add_special_tokens=True,truncation=True,return_tensors = "pt")
  return tokens

def genEmbeddings(tokens):
  model = BertModel.from_pretrained('bert-base-uncased')
  model.eval()
  with torch.no_grad():
    embeddings = model(tokens['input_ids'],tokens['attention_mask'])
  return embeddings

In [15]:
note_descriptions_tokens = genBatchTokens(note_descriptions)
print(note_descriptions_tokens.keys())
print(note_descriptions_tokens['input_ids'].shape)
print(note_descriptions_tokens['attention_mask'].shape)
#print(note_descriptions_tokens)
print(note_descriptions_tokens['input_ids'])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([143, 48])
torch.Size([143, 48])
tensor([[  101,  7733, 13631,  ...,     0,     0,     0],
        [  101,  2317,  3586,  ...,     0,     0,     0],
        [  101,  2339,  2412,  ...,     0,     0,     0],
        ...,
        [  101,  2470,  4796,  ...,     0,     0,     0],
        [  101,  6517,  3070,  ...,     0,     0,     0],
        [  101,  4957,  2000,  ...,     0,     0,     0]])


In [ ]:
note_embeddings = genEmbeddings(note_descriptions_tokens)
print(note_embeddings.keys())
print(note_embeddings['last_hidden_state'].shape)
last_hidden_state = note_embeddings['last_hidden_state']
print(last_hidden_state)

In [25]:
print(last_hidden_state)

tensor([[[-1.1666e+00, -2.3682e-01, -4.2736e-01,  ..., -3.4768e-01,
           1.4737e-01,  2.6906e-01],
         [-3.6302e-01,  1.1102e+00,  5.9645e-01,  ..., -1.9571e-01,
           4.4804e-01, -1.7184e-01],
         [-4.7832e-01,  3.3486e-01,  3.5181e-01,  ..., -6.6172e-01,
           2.0434e-01, -6.0178e-01],
         ...,
         [-5.9421e-01, -3.9434e-02,  3.1631e-01,  ..., -3.9479e-02,
           2.6391e-01,  2.8256e-01],
         [-5.7853e-01, -5.9667e-02,  3.0473e-01,  ..., -3.4475e-02,
           2.3856e-01,  2.7773e-01],
         [-5.5239e-01, -3.2775e-02,  2.9569e-01,  ..., -5.0502e-02,
           2.5698e-01,  2.6989e-01]],

        [[-6.2571e-01, -2.1430e-01, -2.9222e-01,  ..., -1.1877e-01,
           5.5669e-02,  3.1316e-01],
         [-2.6159e-01,  1.5403e-02,  5.2451e-02,  ..., -1.8669e-01,
           2.3264e-02, -2.7804e-01],
         [ 2.9261e-01, -4.9351e-01,  3.0551e-02,  ..., -7.8159e-01,
          -5.6005e-01, -4.8374e-01],
         ...,
         [-3.0209e-01, -1

In [19]:
!pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 8.6 MB/s eta 0:00:00


In [20]:
from pinecone import Pinecone

pc = Pinecone(api_key="c8119aab-b980-40e1-af06-abff6e64a53c")
index = pc.Index("smart-bokx")

In [ ]:
from re import X
for lhs in range(len(last_hidden_state)):
  print(lhs)
  #xdense_embedding = torch.mean(lhs, dim=1).squeeze().numpy()
  #print("Dense vector:", xdense_embedding)

In [27]:
dense_embeddings = torch.mean(last_hidden_state, dim=1).squeeze().numpy()

print("Dense vector:", dense_embeddings)

Dense vector: [[-0.6078871  -0.07106092  0.19682227 ... -0.09283213  0.24800205
   0.24861921]
 [-0.4114375  -0.31500018 -0.3151264  ... -0.15914041  0.15679602
   0.03730177]
 [-0.02879488 -0.13394134  0.11084998 ...  0.02291251  0.34594008
  -0.2149346 ]
 ...
 [-0.2874277  -0.21148483 -0.17862366 ... -0.0578384  -0.22416158
   0.08797896]
 [ 0.16405953 -0.16003384 -0.17841762 ... -0.06285904  0.0317183
  -0.34999394]
 [ 0.1613981  -0.14817823  0.3194487  ... -0.03285281  0.00568046
  -0.26486063]]


In [31]:
def addToPinecons(vec_id,vectors,meta_data):
  user_id = "user1"
  index.upsert(
    vectors=[
        {
            "id": vec_id,
            "values": vectors,
            "metadata": meta_data
        }
    ],
    namespace= user_id
)

In [ ]:
embed_index = 0
for dense_embedding in dense_embeddings:
  #print(dense_embedding)

  print(embed_index)
  meta_data = {
      "note_title": note_titles[embed_index],
      "note_link": note_links[embed_index],
      "note_description": note_descriptions[embed_index]
  }
  addToPinecons("note_"+str(embed_index),dense_embedding.tolist(),meta_data=meta_data)
  embed_index += 1

In [133]:
query_string = "dark history"
gen_query_tokens = genToken(query_string)
query_embedding = genEmbeddings(gen_query_tokens)
query_embeddings = torch.mean(query_embedding['last_hidden_state'], dim=1).squeeze().numpy()

#print(query_embeddings)

In [116]:
def searchPinecone(query_embeddings,top_k):
  user_id = "user1"
  result = index.query(
    namespace=user_id,
    vector=query_embeddings,
    top_k=top_k,
    include_values=True,
    include_metadata=True
  )

  return result

In [134]:
search_result = searchPinecone(query_embeddings.tolist(),5)
#print(search_result)
#

In [135]:
matches = search_result['matches']
for match in matches:
  print(match['metadata']['note_title'])
  print(match['metadata']['note_description'])

https://vm.tiktok.com/ZMraaJnjr/ This post is shared via TikTok Lite. Download TikTok Lite to enjoy more posts:  https://tiktoklite.onelink.me/qItA?pid=txt_ugg
Islam revealed to Mohammed history 
https://vm.tiktok.com/ZMMXJxj8q/
Jesus as a black African
https://vm.tiktok.com/ZMrxfgng3/
Portal to another world opened
https://vm.tiktok.com/ZMrYdddtb/
Empaths and mirror people
https://vm.tiktok.com/ZMMCrb6xv/
Sadnguru talking about heaven and society
